In [ ]:
import re
from os import listdir, makedirs
from os.path import isfile, join
from time import time
import importlib

import constants
import project_logging
import nlp_processor

from constants import *
from project_logging import log
from nlp_processor import NLPProcessor

# set flags to avoid multiple logging handlers
NOTEBOOK = True  # additional StreamHandler disabled if True (should always be true in IPython)
LOG = False  # set to True to write to log file
# LOG_PATH = ...  # overwrite default log file path

In [ ]:
importlib.reload(constants)
importlib.reload(project_logging)
importlib.reload(nlp_processor)

log("##### START #####")


### --- run ---

t0 = time()

LOCAL_PATH = ETL_BASE
FULL_PATH = join(DATA_BASE, LOCAL_PATH)

# filter files for certain prefixes
prefixes = r'^(' + '|'.join(CORPUS_PREFIXES) + r').'
pattern = re.compile(prefixes)
files = sorted([f for f in listdir(FULL_PATH)
                if (isfile(join(FULL_PATH, f)) and pattern.match(f))])
processor = NLPProcessor(spacy_path=DE)

for name in files:
    corpus = re.split(r'\.|_', name)[0]
    fname = join(FULL_PATH, name)
    df = processor.read_process_load(fname, corpus, spacy_to_disk=False, size=1)

t1 = int(time() - t0)
log("finished in {:02d}:{:02d}:{:02d} minutes".format(t1//3600, t1//60, t1 % 60))

df

In [ ]:
print('-----')
processor.read('/home/andreas/workspace/master_cloud/corpora/preprocessed/nlp/OnlineParticipation_nlp.pickle')